<a href="https://colab.research.google.com/github/Raghunandan1234/BERT-Sentiment_Analysis/blob/main/Roberta_Spanish_model_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
es_train = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/es_train.tsv', sep='\t', header=None, usecols=[1,2])
cr_train = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/cr_train.tsv', sep='\t', header=None, usecols=[1,2])
mx_train = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/mx_train.tsv', sep='\t', header=None, usecols=[1,2])
pe_train = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/pe_train.tsv', sep='\t', header=None, usecols=[1,2])
uy_train = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/uy_train.tsv', sep='\t', header=None, usecols=[1,2])

# Concatenate the dataframes
train_df = pd.concat([es_train, cr_train, mx_train, pe_train, uy_train], axis=0, ignore_index=True)
# Removing neutral comments
train_df = train_df[train_df[2] != 'NEU']


# Display data
print('TRAINING DATA\n',train_df.shape)
display(train_df.head())
### VALIDATION DATA ###
# Load the different train dataframes
es_val = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/es_val.tsv', sep='\t', header=None, usecols=[1,2])
cr_val = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/cr_val.tsv', sep='\t', header=None, usecols=[1,2])
mx_val = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/mx_val.tsv', sep='\t', header=None, usecols=[1,2])
pe_val = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/pe_val.tsv', sep='\t', header=None, usecols=[1,2])
uy_val = pd.read_csv('https://raw.githubusercontent.com/lucamarcelo/Vaccine-Tweets-Sentiment-Analysis/main/uy_val.tsv', sep='\t', header=None, usecols=[1,2])
# Concatenate the dataframes
val_df = pd.concat([es_val, cr_val, mx_val, pe_val, uy_val], axis=0, ignore_index=True)
# Removing neutral comments
val_df = val_df[val_df[2] != 'NEU']
# Display data
print('VALIDATION DATA\n',val_df.shape)
display(val_df.head())

TRAINING DATA
 (3279, 2)


,1,2
0,@morbosaborealis jajajaja... eso es verdad... ...,N
3,"Mi última partida jugada, con Sona support. La...",P
4,Tranquilos que con el.dinero de Camacho seguro...,P
5,"@daniacal aún no, pero si estará jugable en el...",P
6,@ragnomuelle Yo a veces hecho de menos mi pelo...,N


VALIDATION DATA
 (1650, 2)


,1,2
0,@chefidiaz no seas muy dura,N
2,Como destrozaba el puto movil ahora mismo,N
4,@omixam no creo que hayan diseñado una tipo pr...,N
5,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,N
6,"@Hanita_chan nfin, no puede ser peor que un mi...",N


In [ ]:
train_df.dropna(axis=0,inplace=True)

In [ ]:
val_df.dropna(axis=0,inplace=True)

In [ ]:
train_texts = list(train_df[1])
train_labels = list(train_df[2])
## Val data
val_texts = list(val_df[1])
val_labels = list(val_df[2])

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer

In [ ]:
 tokenizer = AutoTokenizer.from_pretrained("BSC-TeMU/roberta-base-bne")
# Preprocessing text
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:

d = {'N':0, 'P':1}
## Map the values in the dictionary to the three lists of labels
train_labels = list(pd.Series(train_labels).map(d).astype(int))
val_labels = list(pd.Series(val_labels).map(d).astype(int))
# Create the tensorflow datasets from our encodings
train_dataset = tf.data.Dataset.from_tensor_slices((
dict(train_encodings),train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((
dict(val_encodings),val_labels))

In [ ]:
from transformers import TFAutoModelForSequenceClassification
## Model Definition
roberta_model = TFAutoModelForSequenceClassification.from_pretrained("BSC-TeMU/roberta-base-bne", from_pt=True, num_labels=2)
## Model Compilation
roberta_model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
roberta_model.compile(optimizer=optimizer, loss=loss, metrics=metric)
## Fitting the data
#history = roberta_model.fit(train_dataset, validation_data=val_dataset, epochs=5, batch_size=64)
history = roberta_model.fit(train_dataset.shuffle(len(train_labels)).batch(64), validation_data=val_dataset.shuffle(len(val_labels)).batch(64), epochs=3, batch_size=64)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124052736 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,644,866
Trainable params: 124,644,866
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
52/52 [==============================] - 104s 911ms/step - loss: 0.4447 - sparse_categorical_accuracy: 0.7862 - val_loss: 0.3429 - val_sparse_categorical_accuracy: 0.8552
Epoch 2/3
52/52 [==============================] - 41s 792ms/step - loss: 0.2169 - sparse_categor

In [ ]:
from transformers import pipeline
roberta_classifier = pipeline('sentiment-analysis', model=roberta_model, tokenizer=tokenizer)
# Let's see if it works on an example
roberta_classifier(['Hoy no me encuentro muy bien.', 'Hoy es un día bonito.'])

[{'label': 'LABEL_0', 'score': 0.9951688647270203},
 {'label': 'LABEL_1', 'score': 0.9951258301734924}]

In [ ]:
import matplotlob.pyplot as plt
plt.simproberta